## Business Problem:

Pawdacity is a leading pet store chain in Wyoming with 13 stores throughout the state. This year, Pawdacity would like to expand and open a 14th store. Your manager has asked you to perform an analysis to recommend the city for Pawdacity’s newest store, based on predicted yearly sales.

Your manager has given you the following information to work with:

1- The monthly sales data for all of the Pawdacity stores for the year 2010.

2- NAICS data on the most current sales of all competitor stores where total sales is equal to 12 months of sales.

3- A partially parsed data file that can be used for population numbers.

4- Demographic data (Households with individuals under 18, Land Area, Population Density, and Total Families) for each city and county in the state of Wyoming. For people who are unfamiliar with the US city system, a state contains counties and counties contains one or more cities.


To properly build the model, and select predictor variables, create a dataset with the following columns:

City

2010 Census Population

Total Pawdacity Sales

Households with Under 18

Land Area

Population Density

Total Families



If you’ve done everything correctly, the sum for each of the above columns should be:

Census Population: 213,862

Total Pawdacity Sales: 3,773,304

Households with Under 18: 34,064

Land Area: 33,071

Population Density: 63

Total Families: 62,653

with 11 rows of data



In [62]:
# import the required packages:

import pandas as pd #for data manipulations and analysis
import numpy as np #to support arrays and mathematical functions
import os #allows to interface with the underlying operating system that Python is running on
import matplotlib.pyplot as plt #to create plots
#import sklearn 
from sklearn import linear_model #for machine learning functions 
from sklearn.linear_model import LinearRegression 
from sklearn.cross_validation import train_test_split #For machine learning functions
import statsmodels.api as sm  #for the estimation of many different statistical models
import seaborn as sns #for data visualization
%matplotlib inline 

In [63]:
#What files do we have?
os.listdir()

['.ipynb_checkpoints',
 'p2-2010-pawdacity-monthly-sales-p2-2010-pawdacity-monthly-sales.csv',
 'p2-partially-parsed-wy-web-scrape.csv',
 'p2-wy-453910-naics-data.csv',
 'p2-wy-demographic-data.csv',
 'Pawdacity data preparation (grouping and joining).ipynb']

In [64]:
#loading the files
sales_2010 = pd.read_csv('p2-2010-pawdacity-monthly-sales-p2-2010-pawdacity-monthly-sales.csv')
parsed_web = pd.read_csv('p2-partially-parsed-wy-web-scrape.csv')
demographic = pd.read_csv('p2-wy-demographic-data.csv')

In [65]:
sales_2010

,NAME,ADDRESS,CITY,STATE,ZIP,January,February,March,April,May,June,July,August,September,October,November,December
0,Pawdacity,509 Fort St # A,Buffalo,WY,82834,16200,13392,14688,17064,18360,14040,12960,19224,15984,13392,13176,16848
1,Pawdacity,601 SE Wyoming Blvd Unit 252,Casper,WY,82609,29160,21600,27000,27648,29160,27216,25488,25704,22896,25272,28944,27648
2,Pawdacity,3769 E Lincolnway,Cheyenne,WY,82001,79920,70632,79056,77544,73656,77976,73872,77544,78516,74520,74736,79920
3,Pawdacity,2625 Big Horn Ave,Cody,WY,82414,19440,15984,19008,18144,16632,17496,18792,20304,19224,18144,18576,16632
4,Pawdacity,123 S 2nd St,Douglas,WY,82633,16200,13392,14688,17064,18360,14040,12960,19224,15984,29808,17496,18792
5,Pawdacity,932 Main St,Evanston,WY,82930,24840,21168,21600,22248,24192,24624,25488,25704,22032,21168,25920,24840
6,Pawdacity,200 E Lakeway Rd,Gillette,WY,82718,47520,41796,48384,47088,42336,41904,42120,47088,49032,48168,42984,44712
7,Pawdacity,180 S Bent St,Powell,WY,82435,20520,17928,20304,21168,21600,17928,18144,18576,20304,21168,17496,18792
8,Pawdacity,512 E Main St,Riverton,WY,82501,27000,22032,28512,26784,25920,24192,25056,22896,25488,26352,26784,22248
9,Pawdacity,2706 Commercial Way,Rock Springs,WY,82901,21600,19872,22248,20952,17496,24840,22464,21816,21384,20304,22032,18576


In [66]:
sales_2010.shape

(11, 17)

In [67]:
sales_2010.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 17 columns):
NAME         11 non-null object
ADDRESS      11 non-null object
CITY         11 non-null object
STATE        11 non-null object
ZIP          11 non-null int64
January      11 non-null int64
February     11 non-null int64
March        11 non-null int64
April        11 non-null int64
May          11 non-null int64
June         11 non-null int64
July         11 non-null int64
August       11 non-null int64
September    11 non-null int64
October      11 non-null int64
November     11 non-null int64
December     11 non-null int64
dtypes: int64(13), object(4)
memory usage: 1.5+ KB


In [68]:
#with the sales dataset, we can find the Total Pawdacity Sales and cities
sales_2010 = sales_2010.drop(['NAME','ADDRESS', 'STATE', 'ZIP'], axis = 1)
sales_2010 = sales_2010.groupby('CITY').sum().reset_index()
num_sales = sales_2010.drop('CITY', axis = 1)
sales_2010['Total Pawdacity Sales'] = num_sales.sum(axis = 1)
sales_2010 = sales_2010.filter(['CITY', 'Total Pawdacity Sales'], axis = 1)
sales_2010

,CITY,Total Pawdacity Sales
0,Buffalo,185328
1,Casper,317736
2,Cheyenne,917892
3,Cody,218376
4,Douglas,208008
5,Evanston,283824
6,Gillette,543132
7,Powell,233928
8,Riverton,303264
9,Rock Springs,253584


In [69]:
demographic.head()

,City,County,Land Area,Households with Under 18,Population Density,Total Families
0,Laramie,Albany,2513.745235,2075,5.19,4668.93
1,Rock River,Albany,200.444000,165,0.41,372.30
2,Basin,Big Horn,543.951304,250,0.66,566.43
3,Burlington,Big Horn,137.646214,63,0.17,143.34
4,Byron,Big Horn,252.489592,116,0.31,262.93


In [70]:
#with the demographic dataset, we can find the cities, land ares, Households with Under 18, Population Density
# and Total Families
demographic = demographic.drop('County', axis = 1)
demographic = demographic.rename(columns = {'City': 'CITY'})
demographic = demographic.groupby('CITY').sum().reset_index()
demographic.head()

,CITY,Land Area,Households with Under 18,Population Density,Total Families
0,Afton,853.806650,497,0.93,1040.41
1,Albin,7.685902,37,0.10,74.87
2,Alpine,366.598892,213,0.40,446.72
3,Baggs,253.240322,62,0.06,129.53
4,Bairoil,29.459830,18,0.01,33.70


In [71]:
parsed_web

,City|County,2014 Estimate,2010 Census,2000 Census
0,Afton|Lincoln,"<td>1,968</td>","<td>1,911</td>","<td>1,818</td>"
1,Albin|Laramie,<td>185</td>,<td>181</td>,<td>120</td>
2,Alpine|Lincoln,<td>845</td>,<td>828</td>,<td>550</td>
3,Baggs|Carbon,<td>439</td>,<td>440</td>,<td>348</td>
4,Bairoil|Sweetwater,<td>107</td>,<td>106</td>,<td>97</td>
5,Bar Nunn|Natrona,"<td>2,735</td>","<td>2,213</td>",<td>936</td>
6,Basin ?|Big Horn,"<td>1,312</td>","<td>1,285<sup id=""cite_ref-4"" class=""reference...","<td>1,238</td>"
7,Bear River|Uinta,<td>521</td>,<td>518</td>,<td>-</td>
8,Big Piney|Sublette,<td>538</td>,<td>552</td>,<td>408</td>
9,Buffalo ?|Johnson,"<td>4,615</td>","<td>4,585</td>","<td>3,900</td>"


In [72]:
parsed_web.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
City|County      99 non-null object
2014 Estimate    103 non-null object
2010 Census      103 non-null object
2000 Census      103 non-null object
dtypes: object(4)
memory usage: 3.3+ KB


In [73]:
parsed_web.columns

Index(['City|County', '2014 Estimate', '2010 Census', '2000 Census'], dtype='object')

In [74]:
#let's split the city and county column into two seperate columns
parsed_web['City'], parsed_web['County'] = parsed_web['City|County'].str.split('|', 1).str
parsed_web = parsed_web.drop("City|County", axis = 1)
parsed_web = parsed_web.rename(columns = {'City': 'CITY'})

In [75]:
#let's remove all <td>s from the columns
for val in parsed_web.columns.values:
    parsed_web[val] = parsed_web[val].str.replace("<td>", "")
    parsed_web[val] = parsed_web[val].str.replace("</td>", "")
    #<sup id="cite_ref-5" class="reference"
    parsed_web[val] = parsed_web[val].str.replace("<sup.*?>(.+?)</sup>", "")
    parsed_web[val] = parsed_web[val].str.replace("?", "")

In [76]:
parsed_web

,2014 Estimate,2010 Census,2000 Census,CITY,County
0,"1,968","1,911","1,818",Afton,Lincoln
1,185,181,120,Albin,Laramie
2,845,828,550,Alpine,Lincoln
3,439,440,348,Baggs,Carbon
4,107,106,97,Bairoil,Sweetwater
5,"2,735","2,213",936,Bar Nunn,Natrona
6,"1,312","1,285","1,238",Basin,Big Horn
7,521,518,-,Bear River,Uinta
8,538,552,408,Big Piney,Sublette
9,"4,615","4,585","3,900",Buffalo,Johnson


In [77]:
#let's drop the las 4 rows
parsed_web = parsed_web[:-4]

In [78]:
#now we have to change the type of "2010" census to string
parsed_web['2010 Census'] = parsed_web['2010 Census'].astype(str)
parsed_web['2010 Census'] = parsed_web['2010 Census'].str.replace(",", "")
parsed_web['2010 Census'] = pd.to_numeric(parsed_web['2010 Census'])

/Users/zahrafarhadi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/zahrafarhadi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/zahrafarhadi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [81]:
#now let's group the cities and keep the necessary colums

parsed_web = parsed_web.filter(['CITY', '2010 Census'], axis = 1)
parsed_web = parsed_web.groupby('CITY').sum().reset_index()

In [82]:
parsed_web.head()

,CITY,2010 Census
0,Afton,1911
1,Albin,181
2,Alpine,828
3,Baggs,440
4,Bairoil,106


## Joining the tables

In [88]:
sales_2010.set_index('CITY')
demographic.set_index('CITY')
parsed_web.set_index('CITY')
#display(sales_2010) #city, total padacity sales
#display(demographic) #city, land area, household under 18, population density, total families
#display(parsed_web) #city, 2010 census

table = sales_2010.join(parsed_web.set_index('CITY'), on='CITY')
display(table)
table = table.join(demographic.set_index('CITY'), on='CITY')
display(table)

,CITY,Total Pawdacity Sales,2010 Census
0,Buffalo,185328,NaN
1,Casper,317736,NaN
2,Cheyenne,917892,NaN
3,Cody,218376,NaN
4,Douglas,208008,NaN
5,Evanston,283824,NaN
6,Gillette,543132,NaN
7,Powell,233928,6314.0
8,Riverton,303264,10615.0
9,Rock Springs,253584,23036.0


,CITY,Total Pawdacity Sales,2010 Census,Land Area,Households with Under 18,Population Density,Total Families
0,Buffalo,185328,NaN,3115.507500,746,1.55,1819.50
1,Casper,317736,NaN,3894.309100,7788,11.16,8756.32
2,Cheyenne,917892,NaN,1500.178400,7158,20.34,14612.64
3,Cody,218376,NaN,2998.956960,1403,1.82,3515.62
4,Douglas,208008,NaN,1829.465100,832,1.46,1744.08
5,Evanston,283824,NaN,999.497100,1486,4.95,2712.64
6,Gillette,543132,NaN,2748.852900,4052,5.80,7189.43
7,Powell,233928,6314.0,2673.574550,1251,1.62,3134.18
8,Riverton,303264,10615.0,4796.859815,2680,2.34,5556.49
9,Rock Springs,253584,23036.0,6620.201916,4022,2.78,7572.18


In [84]:
table.describe()

,Land Area,Households with Under 18,Population Density,Total Families,Total Pawdacity Sales
count,99.000000,99.000000,99.000000,99.000000,11.000000
mean,827.837571,582.353535,1.145859,1148.407677,343027.636364
std,1243.493105,1249.994566,2.601399,2183.310426,213538.712215
min,7.685902,2.000000,0.010000,5.790000,185328.000000
25%,100.854266,54.500000,0.095000,114.005000,226152.000000
50%,253.240322,160.000000,0.350000,332.280000,283824.000000
75%,987.769764,404.000000,0.980000,858.695000,312984.000000
max,6620.201916,7788.000000,20.340000,14612.640000,917892.000000


In [85]:
#now let's get the sum and average of the columns
for val in table.columns.values:
    if val != "CITY":
        _sum = table[val].sum()
        _avg = table[val].mean()
        print("{} - sum = {} \t avg = {}".format(val, _sum, _avg))
        #print(table.val)
        #print(val + 'va')

Land Area - sum = 81955.91952966899 	 avg = 827.837571006758
Households with Under 18 - sum = 57653 	 avg = 582.3535353535353
Population Density - sum = 113.44 	 avg = 1.145858585858586
Total Families - sum = 113692.36000000003 	 avg = 1148.4076767676765
Total Pawdacity Sales - sum = 3773304.0 	 avg = 343027.63636363635
